In [1]:
from datetime import datetime
from tkinter import messagebox
import tkinter as tk
import pandas as pd
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore")

In [2]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir('bases/pagos/'):
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(fecha_list[0] + '.' + fecha_list[1] + '.' + fecha_list[2], '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

'03.09.2024'

In [3]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

La carpeta ya existe: reporte\2024.09.04


'reporte\\2024.09.04'

In [4]:
base_pagos_path = 'bases/pagos/Base Pagos ' + fecha + '.xlsx'
asignacion_path = 'bases/asignacion/base_asignacion.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = file_path + '/MONOPRODUCTO ' + fecha + '.xlsx'
multiproducto = file_path + '/MULTIPRODUCTO ' + fecha + '.xlsx'
reactiva = file_path + '/REACTIVA ' + fecha + '.xlsx'
no_enviados = file_path + '/NO ENVIADOS ' + fecha + '.xlsx'
si_enviados = file_path + '/ENVIADOS ' + fecha + '.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)
enviados_path = os.path.abspath(si_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)
print(enviados_path)

bases/pagos/Base Pagos 03.09.2024.xlsx
bases/asignacion/base_asignacion.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.04\MONOPRODUCTO 03.09.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.04\MULTIPRODUCTO 03.09.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.04\REACTIVA 03.09.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.04\NO ENVIADOS 03.09.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.04\ENVIADOS 03.09.2024.xlsx


In [5]:
#################### BASE DE PAGOS ####################

In [6]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(153, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-09-03,28343712,248,235,1331.52,PEN,FABIO SEBASTIAN CASTELLARES ROMERO,P
1,11,2024-09-03,20414784,248,291,819.33,PEN,001102915001612854,P
2,11,2024-09-03,20992021,248,107,356.00,PEN,"CARLOS ALBERTO CHAMBILLA BENSON,,",P
3,11,2024-09-03,27720511,248,113,1300.00,PEN,ADRIANA RUIZ NOLORBE,P
4,11,2024-09-03,30981582,248,143,1114.00,PEN,BETY LUCI CORDOVA ZACARIAS . .,P


In [7]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base = df_base.rename(columns={'CODCEN': 'CC'})

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(153, 10)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FECHA_ENVIO,ID_RESPONSABLE
0,2024-09-03,28343712,00248,235,1331.52,PEN,FABIO SEBASTIAN CASTELLARES ROMERO,P,04-Sep,MIV
1,2024-09-03,20414784,00248,291,819.33,PEN,001102915001612854,P,04-Sep,MIV
2,2024-09-03,20992021,00248,107,356.00,PEN,"CARLOS ALBERTO CHAMBILLA BENSON,,",P,04-Sep,MIV
3,2024-09-03,27720511,00248,113,1300.00,PEN,ADRIANA RUIZ NOLORBE,P,04-Sep,MIV
4,2024-09-03,30981582,00248,143,1114.00,PEN,BETY LUCI CORDOVA ZACARIAS . .,P,04-Sep,MIV


In [8]:
df_base.isnull().sum()

FECHA             0
CC                0
CLAVSERV          0
CENTROPAGO        0
IMPORTE           0
MONEDA            0
NOMBRE            0
ESTADO            0
FECHA_ENVIO       0
ID_RESPONSABLE    0
dtype: int64

In [9]:
#################### ASIGNACION ####################

In [10]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base de asignación?", icon='warning')
if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path)


In [11]:
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

(242511, 9)
codigo               int64
ID_FLAG              int64
cont_18              int64
nombre              object
TIPO_CARTERA        object
TIPO_FONDO          object
clave               object
AGENCIA CORRECTA    object
ID_VTA.             object
dtype: object


,codigo,ID_FLAG,cont_18,nombre,TIPO_CARTERA,TIPO_FONDO,clave,AGENCIA CORRECTA,ID_VTA.
0,30739481,2,1100109600055161,DINA CHOQUE FERNANDEZ,UNSECURED,NaN,BC,MORNESE MORA,-
1,30739481,2,1100109600057679,DINA CHOQUE FERNANDEZ,UNSECURED,NaN,BC,MORNESE MORA,-
2,21874900,4,1100505000360222,JULIA ANGELICA SOTO COTRINA,UNSECURED,NaN,BC,ASESCOM RJ,-
3,30293392,1,1101305002987960,INNOVA FASHION GROUPSAC,UNSECURED,NaN,BC,CLASA MORA,-
4,20687681,1,1101749600365104,ELIZABETH HAYDEE CORDOVA AMAYA,UNSECURED,NaN,BC,RodriguezAngobaldo,-


In [12]:
#################### BACKUPS ####################

In [13]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

Base (153, 10)
Asignacion (242511, 9)


In [14]:
base_count = df_base_test.shape[0]
base_count

153

In [15]:
def actualizar_tipo_cartera(grupo: pd.DataFrame) -> pd.DataFrame:
    if 'UNSECURED' in grupo['TIPO_CARTERA'].values:
        grupo['TIPO_CARTERA'] = 'UNSECURED'
    return grupo

In [16]:
df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)

cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'TIPO_FONDO', 'clave', 'AGENCIA CORRECTA']
df_base_test = df_base_test.merge(df_asignacion_test[cols_asignacion], left_on='CC', right_on='codigo', how='left')
df_base_test = df_base_test.drop(columns=['codigo'])
df_base_test = df_base_test.rename(columns={'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO', 'nombre': 'NOMBRE_CLIENTE', 'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA'})
df_base_test.sort_values(by=['TIPO_CARTERA'], ascending=False, inplace=True)
df_base_test.sort_values(by=['CC'], ascending=True, inplace=True)

filtro = (df_base_test['FLAG'] != 1) & (df_base_test['FLAG'].notna())
df_base_filtro = df_base_test.loc[filtro].groupby('CC').apply(actualizar_tipo_cartera).reset_index(drop=True)
df_base_test = df_base_test.merge(df_base_filtro[['CC', 'TIPO_CARTERA']], on='CC', how='left', suffixes=('', '_y'))
df_base_test['TIPO_CARTERA_FINAL'] = df_base_test['TIPO_CARTERA_y'].fillna(df_base_test['TIPO_CARTERA'])
df_base_test = df_base_test.drop(columns=['TIPO_CARTERA', 'TIPO_CARTERA_y'])
df_base_test = df_base_test.rename(columns={'TIPO_CARTERA_FINAL': 'TIPO_CARTERA'})
#26497246
df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(153, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                Int64
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-03,00996548,00248,222,1050.0,PEN,EDWIN EMILIANO JAUREGUI GONGORA,P,1,001107859600162485,NULL,KSTBC,EDWIN EMILIANO JAUREGUI GONGORA,04-Sep,MIV,UNSECURED,CLASA MORA
1,2024-09-03,03376699,00248,110,2496.0,PEN,VERONICA HILDA HAGUE MARROU - 06722637,P,3,001103525001621897,NULL,EXJ,VERONICA HILDA HAGUE MARROU,04-Sep,MIV,UNSECURED,ASESCOM RJ
10,2024-09-03,04101561,00248,87,2630.0,PEN,SANTOS MORAN CISNERON2,P,1,001102745001992697,NULL,KSTBC,SANTOS MORAN CISNERON,04-Sep,MIV,UNSECURED,CLASA MORA
11,2024-09-03,11176313,00248,324,500.0,PEN,ROCIO DEL PILAR LOPEZ BERTHA,P,1,001108149601126470,NULL,KSTBC,ROCIO DEL PILAR LOPEZ BERTHA,04-Sep,MIV,UNSECURED,CLASA MORA
12,2024-09-03,11216870,00248,333,3700.0,PEN,EDA ISABEL PAUCAR JAUREGUI 06892232,P,1,001103595001034256,NULL,KSTBC,EDA ISABEL PAUCAR JAUREGUI,04-Sep,MIV,UNSECURED,CLASA MORA


In [17]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

REGISTROS OK


In [18]:
df_base_test.shape

(153, 17)

In [19]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
df_base_no_flag.shape

(49, 17)

In [20]:
no_flag_count = df_base_no_flag.shape[0]
no_flag_count

49

In [21]:
flag_count = base_count - no_flag_count
flag_count

104

In [22]:
#################### MONOPRODUCTO ####################

In [23]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
print(df_mono.shape)
df_mono.head(5)

(80, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-03,00996548,00248,222,1050.0,PEN,EDWIN EMILIANO JAUREGUI GONGORA,P,1,001107859600162485,NULL,KSTBC,EDWIN EMILIANO JAUREGUI GONGORA,04-Sep,MIV,UNSECURED,CLASA MORA
10,2024-09-03,04101561,00248,87,2630.0,PEN,SANTOS MORAN CISNERON2,P,1,001102745001992697,NULL,KSTBC,SANTOS MORAN CISNERON,04-Sep,MIV,UNSECURED,CLASA MORA
11,2024-09-03,11176313,00248,324,500.0,PEN,ROCIO DEL PILAR LOPEZ BERTHA,P,1,001108149601126470,NULL,KSTBC,ROCIO DEL PILAR LOPEZ BERTHA,04-Sep,MIV,UNSECURED,CLASA MORA
12,2024-09-03,11216870,00248,333,3700.0,PEN,EDA ISABEL PAUCAR JAUREGUI 06892232,P,1,001103595001034256,NULL,KSTBC,EDA ISABEL PAUCAR JAUREGUI,04-Sep,MIV,UNSECURED,CLASA MORA
14,2024-09-03,13361975,00248,83,1200.0,PEN,DANIA MARIETA SARDON SANCHEZ.....,P,1,001102009600556058,NULL,BC,DANIA MARIETA SARDON SANCHEZ,04-Sep,MIV,SECURED,SIN AGENCIA


In [24]:
mono_count = df_mono.shape[0]
mono_count

80

In [25]:
df_mono['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    76
SECURED       3
NULL          1
Name: count, dtype: int64

In [26]:
df_mono['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        76
REACTIVA     4
Name: count, dtype: int64

In [27]:
df_mono[['TIPO_CARTERA', 'TIPO_FONDO']].value_counts()

TIPO_CARTERA  TIPO_FONDO
UNSECURED     NULL          74
SECURED       REACTIVA       2
UNSECURED     REACTIVA       2
NULL          NULL           1
SECURED       NULL           1
Name: count, dtype: int64

In [28]:
#################### MONO REACTIVA ####################

In [29]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'] != 'NULL')]
df_reactiva.sort_values(by=['CC'], inplace=True)
df_reactiva.reset_index(drop=True, inplace=True)
print(df_reactiva.shape)
df_reactiva.head(5)

(2, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-03,20426327,00248,232,2000.0,PEN,EDWIN SATURNINO CJURO HUARAC 04636799,P,1,001102329600469532,REACTIVA,BC,EDWIN SATURNINO CJURO HUACAC,04-Sep,MIV,UNSECURED,CLASA MORA
1,2024-09-03,21705624,00248,879,70000.0,PEN,TECNA PERU SAC,P,1,001103569600133903,REACTIVA,BC,TECNA PERU SAC,04-Sep,MIV,UNSECURED,ASESCOM RJ


In [30]:
reactiva_count = df_reactiva.shape[0]
reactiva_count

2

In [31]:
df_reactiva.to_excel(reactiva, index=False)

In [32]:
#################### FIN REACTIVA ####################

In [33]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
no_mono_count

4

In [34]:
df_mono = df_mono[df_mono['TIPO_CARTERA'] == 'UNSECURED']
df_mono = df_mono[df_mono['TIPO_FONDO'] == 'NULL']

cols_mono = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA']
df_mono = df_mono[cols_mono]

df_mono.sort_values(by=['CC'], inplace=True)
df_mono.reset_index(drop=True, inplace=True)

print(df_mono.shape)
df_mono.head(5)

(74, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-09-03,00996548,00248,1050.0,PEN,EDWIN EMILIANO JAUREGUI GONGORA,P,1,001107859600162485,EDWIN EMILIANO JAUREGUI GONGORA,04-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA
1,2024-09-03,04101561,00248,2630.0,PEN,SANTOS MORAN CISNERON2,P,1,001102745001992697,SANTOS MORAN CISNERON,04-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA
2,2024-09-03,11176313,00248,500.0,PEN,ROCIO DEL PILAR LOPEZ BERTHA,P,1,001108149601126470,ROCIO DEL PILAR LOPEZ BERTHA,04-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA
3,2024-09-03,11216870,00248,3700.0,PEN,EDA ISABEL PAUCAR JAUREGUI 06892232,P,1,001103595001034256,EDA ISABEL PAUCAR JAUREGUI,04-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA
4,2024-09-03,14839020,00248,200.0,PEN,10279544 - OLGA PEZET OLIVA,P,1,001101905001003212,OLGA NATIVIDAD PEZET OLIVA,04-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA


In [35]:
df_mono.to_excel(monoproducto, index=False)

In [36]:
mono_count_final = df_mono.shape[0]
mono_count_final

74

In [37]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

MONOPRODUCTO OK


In [38]:
#################### FIN MONOPRODUCTO ####################

In [39]:
#################### MULTIPRODUCTO ####################

In [40]:
df_multi = df_base_test[df_base_test['FLAG'] > 1].copy()
print(df_multi.shape)
df_multi.head(5)

(24, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
1,2024-09-03,03376699,00248,110,2496.0,PEN,VERONICA HILDA HAGUE MARROU - 06722637,P,3,001103525001621897,NULL,EXJ,VERONICA HILDA HAGUE MARROU,04-Sep,MIV,UNSECURED,ASESCOM RJ
24,2024-09-03,20597258,00248,368,1000.0,PEN,LARA OTAZU ANA MARIA,P,2,001102619600041719,NULL,EXJ,ANA MARIA LARA OTAZU,04-Sep,MIV,UNSECURED,ASESCOM RJ
34,2024-09-03,21389157,00248,345,50.0,PEN,ANTONIO GUILLERMO TAICO URBINA,P,2,001103675001496814,NULL,KSTBC,ANTONIO GUILLERMO TAICO URBINA,04-Sep,MIV,UNSECURED,CLASA MORA
44,2024-09-03,21916504,00248,272,1795.0,PEN,CHRISTIAN RAFAEL URBINA CARRILO,P,3,001108319601381503,NULL,EXJ,CHRISTIAN RAFAEL URBINA CARRILLO,04-Sep,MIV,UNSECURED,MORNESE MORA
55,2024-09-03,22338444,00248,108,1200.0,PEN,MARCO ANTONIO TEROBEO TAPIA,P,2,001101089600055199,NULL,BC,MARCO ANTONIO TOROBEO TAPIA,04-Sep,MIV,SECURED,SIN AGENCIA


In [41]:
multi_count = df_multi.shape[0]
multi_count

24

In [42]:
df_multi['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    21
SECURED       3
Name: count, dtype: int64

In [43]:
df_multi['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        23
REACTIVA     1
Name: count, dtype: int64

In [44]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
no_multi_count

3

In [45]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

df_multi = df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'])

df_multi.sort_values(by=['CC'], inplace=True)
df_multi.reset_index(drop=True, inplace=True)

print(df_multi.shape)
df_multi.head(5)

(21, 14)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
0,2024-09-03,03376699,00248,2496.00,PEN,VERONICA HILDA HAGUE MARROU - 06722637,P,3,None,None,None,VERONICA HILDA HAGUE MARROU,04-Sep,MIV
1,2024-09-03,20597258,00248,1000.00,PEN,LARA OTAZU ANA MARIA,P,2,None,None,None,ANA MARIA LARA OTAZU,04-Sep,MIV
2,2024-09-03,21389157,00248,50.00,PEN,ANTONIO GUILLERMO TAICO URBINA,P,2,None,None,None,ANTONIO GUILLERMO TAICO URBINA,04-Sep,MIV
3,2024-09-03,21916504,00248,1795.00,PEN,CHRISTIAN RAFAEL URBINA CARRILO,P,3,None,None,None,CHRISTIAN RAFAEL URBINA CARRILLO,04-Sep,MIV
4,2024-09-03,22345922,00248,58.19,PEN,001104085013323046,P,2,None,None,None,FREDY ROLANDOCASIANOLEON,04-Sep,MIV


In [46]:
df_multi.to_excel(multiproducto, index=False)

In [47]:
multi_count_final = df_multi.shape[0]
multi_count_final

21

In [48]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

MULTIPRODUCTO OK


In [49]:
#################### FIN MULTIPRODUCTO ####################

In [50]:
#################### NO ENVIADOS ####################

In [51]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

No enviados: 56
Enviados: 97
Total: 153


In [52]:
if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

REGISTROS OK


In [53]:
print(df_no_mono.columns)
print(df_no_multi.columns)
print(df_base_no_flag.columns)

Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')
Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')
Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')


In [54]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].astype('Int64').fillna(0)

df_no_enviados['CONTRATO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['CONTRATO'], axis=1)
df_no_enviados['TIPO_FONDO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_FONDO'], axis=1)
df_no_enviados['TIPO_CARTERA'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_CARTERA'], axis=1)

df_no_enviados['FLAG'] = df_no_enviados['FLAG'].apply(lambda x: '' if x == 0 else x)

df_no_enviados.sort_values(by=['CC'], inplace=True)
df_no_enviados.reset_index(drop=True, inplace=True)

print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

(56, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG               object
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-03,11224490,00248,250,1385.00,PEN,JUAN MILTON MUÑOZ RIOS/26684543 TC,P,,None,None,NaN,NaN,04-Sep,MIV,None,NaN
1,2024-09-03,13361975,00248,83,1200.00,PEN,DANIA MARIETA SARDON SANCHEZ.....,P,1,001102009600556058,NULL,BC,DANIA MARIETA SARDON SANCHEZ,04-Sep,MIV,SECURED,SIN AGENCIA
2,2024-09-03,14353666,00248,247,12586.97,PEN,EXT JUD 001102475001818843,P,,None,None,NaN,NaN,04-Sep,MIV,None,NaN
3,2024-09-03,20414784,00248,291,819.33,PEN,001102915001612854,P,,None,None,NaN,NaN,04-Sep,MIV,None,NaN
4,2024-09-03,21236576,00248,275,200.00,PEN,"MIGUEL GARRO ZULOAGA,",P,,None,None,NaN,NaN,04-Sep,MIV,None,NaN


In [55]:
if no_eviados == df_no_enviados.shape[0]:
    print('NO ENVIADOS OK')
else:
    print('NO ENVIADOS ERROR')

NO ENVIADOS OK


In [56]:
df_no_enviados.to_excel(no_enviados, index=False)

In [57]:
#################### FORMATO EXCEL ####################

In [58]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 15):  # Columnas L-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi_final':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 16):  # Columnas L-P
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'instruccion':
        for col in range(1, 11): # Columnas A-J
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(11, 13): # Columnas K-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 17):  # Columnas M-P
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(17, 18):  # Columna Q
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

In [59]:
format_excel(mono_path, "mono")
format_excel(multi_path, "multi")
format_excel(react_path, "instruccion")
format_excel(no_enviados_path, "instruccion")

os.startfile(mono_path)
os.startfile(multi_path)
os.startfile(react_path)        
os.startfile(no_enviados_path)

In [60]:
#################### AGENCIAS ####################

In [61]:
x

NameError: name 'x' is not defined

In [ ]:
df_1 = pd.read_excel(file_path+'/multi_agencias/RJ.xlsx')
df_2 = pd.read_excel(file_path+'/multi_agencias/CLASA.xlsx')
df_3 = pd.read_excel(file_path+'/multi_agencias/MORNESE.xlsx')


df_1 = df_1[cols_multi]
df_2 = df_2[cols_multi]
df_3 = df_3[cols_multi]

df_1['AGENCIA'] = 'ASESCOM RJ'
df_2['AGENCIA'] = 'CLASA MORA'
df_3['AGENCIA'] = 'MORNESE MORA'

In [ ]:
df_agencias = pd.concat([df_1, df_2, df_3])
df_agencias.dropna(subset=['CONTRATO'], inplace=True)
df_agencias['CONTRATO'] = df_agencias['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if len(x)==18 else None)
df_agencias.dropna(subset=['CONTRATO'], inplace=True)
df_agencias.reset_index(drop=True, inplace=True)

print(df_agencias.shape)
print(df_agencias.dtypes)
df_agencias.head(5)

In [ ]:
df_multi = pd.read_excel(multi_path)
df_multi['AGENCIA'] = 'NULL'
print(df_multi.shape)
df_multi.head(5)

In [ ]:
df_multi_final = pd.concat([df_multi, df_agencias])

df_multi_final['CC'] = df_multi_final['CC'].astype('Int64').astype(str).str.zfill(8)
df_multi_final.drop(columns=['TIPO_FONDO', 'CARTERA'], inplace=True)
df_multi_final.sort_values(by=['CC', 'CONTRATO'], inplace=True)

df_multi_final.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)
df_multi_final.reset_index(drop=True, inplace=True)

df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['cont_18'] = df_asignacion_test['cont_18'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_asignacion = ['cont_18', 'TIPO_FONDO', 'clave', 'TIPO_CARTERA', 'AGENCIA CORRECTA']
df_multi_final = df_multi_final.merge(df_asignacion_test[cols_asignacion], left_on='CONTRATO', right_on='cont_18', how='left')

df_multi_final = df_multi_final.drop(columns=['cont_18'])

df_multi_final = df_multi_final.rename(columns={'clave': 'CARTERA'})

df_multi_final['CONTRATO'] = df_multi_final['CONTRATO'].fillna('NULL')
df_multi_final['TIPO_FONDO'] = df_multi_final['TIPO_FONDO'].fillna('NULL')
df_multi_final['CARTERA'] = df_multi_final['CARTERA'].fillna('NULL')
df_multi_final['TIPO_CARTERA'] = df_multi_final['TIPO_CARTERA'].fillna('NULL')
df_multi_final['AGENCIA CORRECTA'] = df_multi_final['AGENCIA CORRECTA'].fillna('NULL')
df_multi_final['CLAVSERV'] = df_multi_final['CLAVSERV'].astype(str).str.zfill(5)

df_multi_final['AGENCIA CORRECTA'] = df_multi_final.apply(lambda row: row['AGENCIA CORRECTA'] if row['AGENCIA CORRECTA'] == row['AGENCIA'] else f"{row['AGENCIA']}/{row['AGENCIA CORRECTA']}", axis=1)

df_multi_final = df_multi_final.drop(columns=['AGENCIA'])
df_multi_final.rename(columns={'AGENCIA CORRECTA': 'AGENCIA'}, inplace=True)

cols_multi_final = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_multi_final = df_multi_final[cols_multi_final]
df_multi_final.reset_index(drop=True, inplace=True)

print(df_multi_final.shape)
print(df_multi_final.dtypes)
df_multi_final.head(5)

In [ ]:
df_no_enviados_multi = df_multi_final[(df_multi_final['CONTRATO'] == 'NULL') | (df_multi_final['TIPO_CARTERA'] != 'UNSECURED')]
df_no_enviados_multi

In [ ]:
df_reactivas_multi = df_multi_final[(df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & (df_multi_final['TIPO_FONDO'] != 'NULL')]
df_reactivas_multi

In [ ]:
df_multi_final = df_multi_final[(df_multi_final['CONTRATO'] != 'NULL') & (df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & (df_multi_final['TIPO_FONDO'] == 'NULL')]
df_multi_final.to_excel(multiproducto, index=False)

print(df_multi_final.shape)
df_multi_final.head(5)

In [ ]:
df_no_enviados_final = pd.concat([df_no_enviados, df_no_enviados_multi])

df_no_enviados_final['CONTRATO'] = df_no_enviados_final['CONTRATO'].fillna('NULL')
df_no_enviados_final['TIPO_FONDO'] = df_no_enviados_final['TIPO_FONDO'].fillna('NULL')
df_no_enviados_final['CARTERA'] = df_no_enviados_final['CARTERA'].fillna('NULL')
df_no_enviados_final['TIPO_CARTERA'] = df_no_enviados_final['TIPO_CARTERA'].fillna('NULL')
df_no_enviados_final['NOMBRE_CLIENTE'] = df_no_enviados_final['NOMBRE_CLIENTE'].fillna('NULL')
df_no_enviados_final['AGENCIA'] = df_no_enviados_final['AGENCIA'].fillna('NULL')
df_no_enviados_final['CLAVSERV'] = df_no_enviados_final['CLAVSERV'].apply(lambda x: str(int(x)).zfill(5))

df_no_enviados_final.to_excel(no_enviados, index=False)

print(df_no_enviados_final.shape)
df_no_enviados_final.head(5)

In [ ]:
df_reactiva_final = pd.concat([df_reactiva, df_reactivas_multi])
df_reactiva_final.to_excel(reactiva, index=False)

print(df_reactiva_final.shape)
df_reactiva_final.head(5)

In [ ]:
df_enviados = pd.concat([df_mono, df_multi_final])

print(df_enviados.shape)
df_enviados.head(5)

In [ ]:
df_base_test_final = df_base_test[['CC', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE']]

df_base_test_final['KEY'] = df_base_test_final['CC'].astype(str) + df_base_test_final['IMPORTE'].astype(str) + df_base_test_final['MONEDA'] + df_base_test_final['NOMBRE']
df_enviados['KEY'] = df_enviados['CC'].astype(str) + df_enviados['IMPORTE'].astype(str) + df_enviados['MONEDA'] + df_enviados['NOMBRE']

df_enviados_final = df_enviados.merge(df_base_test_final[['KEY', 'CENTROPAGO']], on='KEY', how='left')
df_enviados_final.drop(columns=['KEY'], inplace=True)
df_enviados_final['CLAVSERV'] = df_enviados_final['CLAVSERV'].apply(lambda x: str(int(x)).zfill(5))
df_enviados_final.sort_values(by=['FLAG','CC'], inplace=True)
df_enviados_final.reset_index(drop=True, inplace=True)

cols_enviados = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_enviados_final = df_enviados_final[cols_enviados]
df_enviados_final.to_excel(si_enviados, index=False)

print(df_enviados_final.shape)
df_enviados_final.head(5)

In [ ]:
format_excel(multi_path, "multi_final")
format_excel(react_path, "instruccion")
format_excel(no_enviados_path, "instruccion")
format_excel(enviados_path, "instruccion")

os.startfile(multi_path)
os.startfile(react_path)        
os.startfile(no_enviados_path)
os.startfile(enviados_path)